# Setting Limits
1) Make cuts on $E_e$
2) Make cuts on background (1 shower, 0 tracks)
3) Make cuts on $\cos(\theta_\text{NuMI})$ for both
4) Set limits
    1) For the fixed $M_\chi/M_{A'}$ case
    2) For the fixed $M_\chi$ case
    

In [10]:
import os
import uproot3 as up3
import uproot
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pyhf
import sys
sys.path.append("../")
#from src import *
from src.data import load_root_file
from src.transforms import add_cos_theta_signal
from src.PoT_scalling import pot_scale

dm_types = ["scalar", "fermion"]
RUN1_MICROBOONE_POT = 2e20

In [2]:
# Cuts
E_e_min = 0.1 # GeV from ...
cos_theta_numi_cuts = [0.55, 0.85] # should probably study this AFTER E_e cut

In [3]:
def two_bin_hist(df : pd.DataFrame, pot_scale : float, cos_theta_numi_cuts : list):
    """Generate 2 bin histogram for the given data frame
    bin1 = cut gives background
    bin2 = cut gives signal

    Args:
        df (pandas.DataFrame): dataframe containing a column with "cos_theta_NuMI
        pot_scale (float): pmax * pot_run_1 / pot_sample
        cos_theta_numi_cuts (list): [cut lower, cut upper] (such that we get signal with these cuts)

    Returns:
        (np.array, np.array) : [counts background, counts signal], [bin edges]
    """
    

    hist = np.zeros(2)
    hist[0] = pot_scale * len(df[(df["cos_theta_numi"] < cos_theta_numi_cuts[0]) | (df["cos_theta_numi"] > cos_theta_numi_cuts[1])])
    hist[1] = pot_scale * len(df[(df["cos_theta_numi"] >= cos_theta_numi_cuts[0]) & (df["cos_theta_numi"] <= cos_theta_numi_cuts[1])])
    return hist, np.array([0,0.5,1])

### Backgrounds

In [4]:
CUT_SHOWERS = 1
CUT_TRACKS = 0

vars = ['reco_asso_showers', 'reco_asso_tracks', 'reco_shower_dirx', 'reco_shower_diry', 'reco_shower_dirz', 'reco_shower_energy_max']
bkg_path = "../data/root/background/"

def cuts(df):
    # for now, 0 tracks, 1 shower
    df = df[df['reco_asso_showers'] == CUT_SHOWERS]
    df = df[df['reco_asso_tracks'] == CUT_TRACKS]
    return df


bkg_incryo_df = pd.concat([load_root_file("run1_NuMI_nu_overlay_with_weights.root", bkg_path, vars, cuts, 10_000, 'vertex_tree;269'), 
                          load_root_file("run1_NuMI_nu_overlay_with_weights.root", bkg_path, vars, cuts, 10_000, 'vertex_tree;268')])
bkg_dirt_df = pd.concat([load_root_file("run1_NuMI_dirt_with_weights.root", bkg_path, vars, cuts, 10_000, 'vertex_tree;127'),
                        load_root_file("run1_NuMI_dirt_with_weights.root", bkg_path, vars, cuts, 10_000, 'vertex_tree;126')])
bkg_beamoff_df = load_root_file("run1_NuMI_offbeam_full_set_sp.root", bkg_path, vars, cuts, 10_000, "singlephotonana")

bkg_total = pd.concat([bkg_incryo_df, bkg_dirt_df, bkg_beamoff_df])

/home/alex/projects/DarkElectronScatter/notebooks/../src/data.py:53: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat((df, df_temp))
/home/alex/projects/DarkElectronScatter/notebooks/../src/data.py:53: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat((df, df_temp))
/home/alex/projects/DarkElectronScatter/notebooks/../src/data.py:53: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer e

In [5]:
# get the 2-bin histogram for the background
# pot_run1 / pot_file
bkg_pot_df = pd.DataFrame(columns=["bkg_type", "pot_ratio"])
bkg_pot_df.loc[0] = ["nu",2.35e+21/2.2e+20]
bkg_pot_df.loc[1] = ["dirt",1.55e+21/2.2e+20]
bkg_pot_df.loc[2] = ["offbeam",0.98*(5736147/9186361.390000)]
bkg_hist_incryo, bkg_edges_incryo = two_bin_hist(bkg_incryo_df, bkg_pot_df[bkg_pot_df["bkg_type"]=="nu"]["pot_ratio"], cos_theta_numi_cuts)
bkg_hist_dirt, bkg_edges_dirt = two_bin_hist(bkg_dirt_df, bkg_pot_df[bkg_pot_df["bkg_type"]=="dirt"]["pot_ratio"], cos_theta_numi_cuts)
bkg_hist_beamoff, bkg_edges_beamoff = two_bin_hist(bkg_beamoff_df, bkg_pot_df[bkg_pot_df["bkg_type"]=="offbeam"]["pot_ratio"], cos_theta_numi_cuts)

bkg_hist_total = bkg_hist_incryo + bkg_hist_dirt + bkg_hist_beamoff
bkg_edges_total = bkg_edges_incryo

/tmp/ipykernel_920/3949741570.py:17: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  hist[0] = pot_scale * len(df[(df["cos_theta_numi"] < cos_theta_numi_cuts[0]) | (df["cos_theta_numi"] > cos_theta_numi_cuts[1])])
/tmp/ipykernel_920/3949741570.py:18: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  hist[1] = pot_scale * len(df[(df["cos_theta_numi"] >= cos_theta_numi_cuts[0]) & (df["cos_theta_numi"] <= cos_theta_numi_cuts[1])])


### Fixed $M_\chi/M_{A'}$

In [7]:
data_dir = "../data/root/BdNMC/"

n_evts_df = []

for dm_type in dm_types:
    for file in os.listdir(data_dir+dm_type+"/"):
        with up3.open(data_dir+dm_type+"/"+file) as f:
            if f.keys() == []:
                continue

            df = f["elecron_tree"].pandas.df()
            df = df[df["electron_energy"] >= 0.1] # Cut on Energy
            n_electrons = len(df)
            df = add_cos_theta_signal(df)
            # df = df[df["cos_theta_numi"] >= cos_theta_numi_cuts[0]]
            # df = df[df["cos_theta_numi"] <= cos_theta_numi_cuts[1]]
            # this is the signal events.
            # n_electrons_after_cuts = len(df)
            pot = f["pot_tree"]["tot_pot"].array()
            pmax = f["pot_tree"]["pmax"].array()
            # signal_bin =  pot_scale(n_electrons_after_cuts, pot, RUN1_MICROBOONE_POT) * pmax
            # background_bin = pot_scale((n_electrons - n_electrons_after_cuts), pot, RUN1_MICROBOONE_POT) * pmax


            dt = file.split("_")[5].strip(".root")
            ma = file.split("_")[3]
            meson_t = file[0:3]

            pot_sc = pot_scale(n_electrons, pot, RUN1_MICROBOONE_POT) * pmax
            hist, edges = two_bin_hist(df, pot_sc, cos_theta_numi_cuts)
            n_evts_df.append([dm_type, dt, ma, meson_t, hist, edges])

n_evts_df = pd.DataFrame(n_evts_df, columns=["dm_type", "dt", "ma", "meson_t", "hist", "edges"])
#display(n_evts_df)


# Need to add pi0 and eta files together

get_other_meson_dict = {"pi0":"eta", "eta":"pi0"}

merged_df = pd.DataFrame(columns=["dm_type", "dt", "ma", "hist", "edges"])

i = 0
for signal in n_evts_df.iterrows():
    # okay so we cannot assume we always have a pi0 and eta file
    # so we need to check if the other file exists
    dt = signal[1]["dt"]
    ma = signal[1]["ma"]
    meson_t = signal[1]["meson_t"]
    dm_type = signal[1]["dm_type"]
    other_meson_t = get_other_meson_dict[meson_t]
    
    # look for the other meson in the n_evts_df, add it, and remove it
    other_meson = n_evts_df[(n_evts_df["meson_t"]==other_meson_t) & (n_evts_df["dm_type"]==dm_type) & (n_evts_df["dt"]==dt) & (n_evts_df["ma"]==ma)]
    if len(other_meson) == 0:
        # no matching meson found
        merged_df.loc[i] = signal[1]
        i += 1
    else:
        # add the two histograms together
        hist = signal[1]["hist"] + other_meson["hist"].values[0]
        edges = signal[1]["edges"]
        merged_df.loc[i] = [dm_type, dt, ma, hist, edges]
        n_evts_df.drop(other_meson.index, inplace=True) # to avoid duplicates in the merged_df
        i += 1

/tmp/ipykernel_920/3949741570.py:17: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  hist[0] = pot_scale * len(df[(df["cos_theta_numi"] < cos_theta_numi_cuts[0]) | (df["cos_theta_numi"] > cos_theta_numi_cuts[1])])
/tmp/ipykernel_920/3949741570.py:18: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  hist[1] = pot_scale * len(df[(df["cos_theta_numi"] >= cos_theta_numi_cuts[0]) & (df["cos_theta_numi"] <= cos_theta_numi_cuts[1])])
/tmp/ipykernel_920/3949741570.py:17: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing thi

In [12]:
plotting = False
for signal in merged_df.iterrows():
    dt = signal[1]["dt"]
    dm = signal[1]["dm_type"]
    ma = signal[1]["ma"]
    if plotting:
        fig = plt.figure()
        ax = fig.add_subplot(111)
        ax.stairs(bkg_hist_total, bkg_edges_total, label="Background", color="tab:red")
        ax.stairs(signal[1]["hist"], signal[1]["edges"], label="Signal", color="tab:blue")
        ax.legend()
        ax.set(title=r"$M_\chi/M_{A'}$="+signal[1]["dt"]+r" $M_{A'}="+signal[1]["ma"]+"$ GeV", xlabel=r"Signal Score", ylabel="Events")
        plt.savefig("../plots/signal_background_scores/fixed_dt/"+f"{dm}_ma_{ma}_dt_{dt}.png")
        plt.savefig("../plots/signal_background_scores/fixed_dt/"+f"{dm}_ma_{ma}_dt_{dt}.pdf")
        plt.show()
    
    # add the histograms to root files
    with uproot.recreate("../data/root/hist/fixed_dt/"+f"{dm}_ma_{ma}_dt_{dt}.root") as f:
        f["signal"] = signal[1]["hist"], signal[1]["edges"]
        f["background"] = bkg_hist_total, bkg_edges_total


In [ ]:
x = np.ones(3)
print(list(x))

[1.0, 1.0, 1.0]


In [ ]:
# set limits

# first add 2-bin histograms to root files
root_hist_path = "../data/root/hist/"

# pot scale background


### Fixed $M_\chi$
1) Read the .out files to check for line "in Off-Shell mode" or "in On-Shell mode"
2) for now, ignore "on shell" files (we dont know that they are handelled correctly)
3) Set Limits

In [ ]:
# Get list of data files
data_dir = "../data/root/BdNMC/fixed_Mchi/"
fnames = []
for dmt in dm_types:
    for file in os.listdir(data_dir+"/"+dmt+"/"):
        if file.endswith(".root"):
            meson = file.split("_")[0]
            ma = file.split("_")[3]
            mchi = file.split("_")[5]
            fnames.append((dmt, meson, ma, mchi, file, True))

fnames = pd.DataFrame(fnames, columns=["dm_type", "meson", "ma", "mchi", "file", "Off Shell"])

# go through the .out files
# e.g nodefixed_Mchi__pi0_scalar_ma_0.09_mchi_0.001.out
out_path = "../data/out/fixed_Mchi/"
for file in fnames:
    try:
        with open(out_path+"nodefixed_Mchi__") as f:
            for line in f:
                if "Off-Shell" in line:
                    print(line)
                    # FINISH THIS!
                elif "On-Shell" in line:
                    print(file, line)
                    fnames[file, "Off Shell"] = False # idk if this works need to check later
    except:
        print("File not found", file)
